<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       PowerTransform function in Vantage
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>Introduction</b></p>

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>PowerTransform</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The PowerTransform() function takes a time series or numerically-sequenced series and applies a power transform to the series to produce a one-dimensional array. The passed-in series can be either a univariate or multivariate series. This function is useful for transforming an input series that has heteroscedastic  variance into a result series that has homoscedastic variance.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following procedure is an example of how to get forecast values for a heteroscedastic time series using PowerTransform() function:</p>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Apply PowerTransform() function to the heteroscedastic time series.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Use the resulting homoscedastic time series to build an ARIMA forecasting model.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Use the model to produce the initial forecast of the homoscedastic time series.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Use the backward transform on the initial forecast to extract the forecast values for the heteroscedastic time series.</li>
</p>

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>1. Initiate a connection to Vantage</b>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In the section, we import the required libraries and set environment variables and environment paths (if required).

In [ ]:
from teradataml import (
    create_context,
    execute_sql,
    load_example_data,
    DataFrame,
    in_schema,
    TDSeries,
    PowerTransform,
    remove_context,
    copy_to_sql,
    db_drop_table
    )

from teradatasqlalchemy.types import *

from teradataml import to_numeric
# Modify the following to match the specific client environment settings
display.max_rows = 5

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>1.1 Connect to Vantage</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>You will be prompted to provide the password. Enter your password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../../UseCases/startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=PP_PowerTransform_Python.ipynb;' UPDATE FOR SESSION; ''')

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Begin running steps with Shift + Enter keys. </p>

<hr style='height:1px;border:none;background-color:#00233C;'>

<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>1.2 Getting Data for This Demo</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Here, we will get the time series data which is available in the teradataml library and use the same to show the usage of the function.</p>

In [ ]:
load_example_data("uaf", ["production_data"])

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>2. Data Exploration</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Create a "Virtual DataFrame" that points to the data set in Vantage. Check the shape of the dataframe as check the datatype of all the columns of the dataframe.</p>

In [ ]:
data = DataFrame.from_table("production_data")
data

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>3. PowerTransform</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The function returns the results, which transforms heteroscedastic time series to homoscedastic time series.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The first step is to convert the data into TDSeries, which is required for the input time series which are passed to the PowerTransform function.</p>


In [ ]:
data_series_df = TDSeries(data=data,
                              id="product_id",
                              row_index="TD_TIMECODE",
                              payload_field=["beer_sales", "wine_sales"],
                              payload_content="MULTIVAR_REAL")

In [ ]:
uaf_out = PowerTransform(data=data_series_df,
                             back_transform=True,
                             p=0.0,
                             b=0.0,
                             lambda1=0.5)
powertransform_df = uaf_out.result
powertransform_df

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>4. Cleanup</b>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following code will clean up tables created above.</p>

In [ ]:
db_drop_table("production_data")

In [ ]:
remove_context()

<hr style="height:1px;border:none;background-color:#00233C;">

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Teradataml Python reference: <a href = 'https://docs.teradata.com/search/all?query=Python+Package+User+Guide&content-lang=en-US'>here</a></li>
    <li>PowerTransform function reference: <a href = 'https://docs.teradata.com/search/all?query=PowerTransform&content-lang=en-US'>here</a></li>
    
</ul>

<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #00233C">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2025. All Rights Reserved
        </div>
    </div>
</footer>